# Servir predições do modelo para o banco de dados

    - Do banco de dados criar o dashboard no Power BI para o cliente consumir as predições do modelo
    - A tabela final será salva em uma outra tabela que vai se chamar Supermarket_Silver sendo assim vai ser derivada da Gold.
    - 

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os

import mlflow
import mlflow.sklearn

from dotenv import load_dotenv

In [2]:
# Variáveis de ambiente
dotenv_path = r"C:\Users\erico\Documents\projeto-regressao\Supermarket-store-sales-prediction\.env\.env"
load_dotenv(dotenv_path)

True

In [3]:
# Utils
def mean_absolute_percentage_error(y_true, yhat):
    return np.mean(np.abs((y_true - yhat) / y_true))

def mean_absolute_error(y_true, yhat):
    return np.mean(np.abs(y_true - yhat))

def root_mean_squared_error(y_true, yhat):
    return np.sqrt(np.mean(np.square(y_true - yhat)))


def calculate_model_revenue(df:pd.DataFrame):
    """
    Calculates the return on investment (ROI) of a model relative to the mean baseline.

    Args:
        df : pandas.DataFrame, a dataframe containing the model's data.

    Returns:
        - "baseline $": the return on the mean baseline in dollars.
        - "model $": the return on the model in dollars.
        - "diff Model/Baseline": the difference between the sum of sales predicted by the model and the baseline, indicating how much more accurately the model predicted sales compared to the baseline.

    This function calculates the ROI of a model by comparing it to a mean baseline. It takes in a pandas DataFrame containing the model's data and returns the return on the baseline, the return on the model, and the difference between the two. The function also prints out the results in a formatted way for easy interpretation. The output will tell whether the model performed better or worse than the baseline, and by how much. 

        """
    # Calcula o retorno do modelo em relação ao baseline de média
    baseline_return = df['average_store_sales'].sum() - df['store_sales'].sum()
    model_return = df['predict_store_sales'].sum() - df['store_sales'].sum()
    diff_model_baseline = (model_return - baseline_return)
    diff_percent = (diff_model_baseline / baseline_return) * 100
    diff_percent_model_store_sales = (model_return / df['store_sales'].sum()) * 100
    diff_percent_baseline_store_sales = (baseline_return / df['store_sales'].sum()) * 100

    # Formata os resultados para exibição
    baseline_return_formatted = f'${baseline_return:,.2f}'
    model_return_formatted = f'${model_return:,.2f}'
    diff_model_baseline_formatted = f'${diff_model_baseline:,.2f}'

    print(f'O baseline de média previu {baseline_return_formatted} em relação as vendas reais.')
    print(f'O modelo previu {model_return_formatted} em relação as vendas reais.')
    print(f'A diferença percentual entre o modelo e as vendas reais são de {(diff_percent_model_store_sales):.2f}%')
    print(f'A diferença percentual entre o baseline de média e as vendas reais são de {(diff_percent_baseline_store_sales):.2f}%')

    if model_return > baseline_return:
        print(f'O modelo consegue recuperar {diff_model_baseline_formatted} que eram perdidos ao usar a média como previsão de vendas!')
    elif model_return == baseline_return:
        print(f'O modelo obteve o mesmo resultado {diff_model_baseline_formatted} que era previsto ao usar a média como previsão de vendas!')
    else:
        print(f'O modelo obteve resultado pior que usar a média como previsão de vendas {diff_model_baseline_formatted} !')

    return None

# Extração Supermarket Gold - Tabela FATO

In [4]:
# Conectar ao banco de dados
conn = psycopg2.connect(
    host="localhost",
    database="Supermarket",
    user="postgres",
    password=os.environ.get('PG_PASSWORD')
)

In [5]:
# Abrir um cursor para executar consultas
cur = conn.cursor()
cur.execute("ROLLBACK")

In [6]:
# Consultar o banco de dados e obter média das store_sales excluindo os últimos 180 dias que serão previstos pelo modelo de regressão linear

query = """ SELECT AVG(store_sales) AS avg_sales
            FROM Supermarket_Gold
            WHERE sales_date < NOW() - INTERVAL '180 DAY'
            AND store_id NOT IN (
                                SELECT store_id
                                FROM Supermarket_Gold
                                WHERE sales_date >= NOW() - INTERVAL '180 DAY'
                                );
        """

cur.execute(query)
mean_sales_30 = cur.fetchone()[0]

In [7]:
# Média de vendas que será usada pelos analistas para prever as próximas vendas do mês que vem que é o teste que vai ser previsto pelo modelo
int(mean_sales_30)

58812

In [8]:
# Consultar o banco de dados e criar o DataFrame pandas
# Definir registros para predição
# Últimos 180 store_id (180 dias)
# Simulando novos registros 

query = """ SELECT * FROM Supermarket_Gold 
            ORDER BY store_id 
            DESC LIMIT 180; 
        """

df = pd.read_sql_query(query, conn)

C:\Users\erico\AppData\Local\Temp\ipykernel_132016\1594901201.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [9]:
# Ver linhas do conjunto para previsão 
df

,store_id,sales_date,store_area,items_available,daily_customer_count,store_sales
0,896,2023-06-15,1174,1429,1110,54340
1,895,2023-06-14,1299,1560,770,96610
2,894,2023-06-13,1200,1436,1060,76440
3,893,2023-06-12,1387,1663,850,82080
4,892,2023-06-11,1582,1910,1080,66390
...,...,...,...,...,...,...
175,721,2022-12-22,1118,1338,1040,40450
176,720,2022-12-21,1827,2160,930,36160
177,719,2022-12-20,1430,1743,970,53230
178,718,2022-12-19,1582,1894,1100,41800


In [10]:
df.shape

(180, 6)

# Mlflow - carregar modelos registrados com melhor performance para predição dos novos dados

In [11]:
# Config Mlflow
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [12]:
# Kmeans
logged_model_cluster = 'runs:/b890ca82406845518c440af7491028c4/Kmeans_12'

# Load model as a PyFuncModel.
km_model = mlflow.pyfunc.load_model(logged_model_cluster)

In [13]:
# Regressão Linear 

logged_model_sales = 'runs:/0951c733113245be85b36d06bf882950/linear_regression_opt_roi'
# Load model as a PyFuncModel.
lr_model = mlflow.pyfunc.load_model(logged_model_sales)

# Criação de variáveis
    - Aplicar modelo de clusterização para criar a coluna store_cluster

In [14]:
# Prever cluster
cluster = km_model.predict(df.drop(['store_id','sales_date','store_sales'], axis=1))
# Adicionar labels ao conjunto de teste
df['store_cluster'] = cluster

In [15]:
df.head()

,store_id,sales_date,store_area,items_available,daily_customer_count,store_sales,store_cluster
0,896,2023-06-15,1174,1429,1110,54340,11
1,895,2023-06-14,1299,1560,770,96610,1
2,894,2023-06-13,1200,1436,1060,76440,11
3,893,2023-06-12,1387,1663,850,82080,2
4,892,2023-06-11,1582,1910,1080,66390,6


# Prevendo novos dados 

In [16]:
# Separar variáveis independentes e dependente
X_test = df.drop(['sales_date','store_sales','daily_customer_count'], axis=1)
y_test = df['store_sales']

In [17]:
# prevendo com modelo de regressão linear
y_pred = lr_model.predict(X_test)

In [18]:
# adiciona previsão do modelo a base de teste original
df['predict_store_sales'] = y_pred

In [19]:
# adiciona previsão de média do baseline a base de teste original
df['average_store_sales'] = int(mean_sales_30)

In [20]:
df.head()

,store_id,sales_date,store_area,items_available,daily_customer_count,store_sales,store_cluster,predict_store_sales,average_store_sales
0,896,2023-06-15,1174,1429,1110,54340,11,58334.967015,58812
1,895,2023-06-14,1299,1560,770,96610,1,59284.845700,58812
2,894,2023-06-13,1200,1436,1060,76440,11,58385.795468,58812
3,893,2023-06-12,1387,1663,850,82080,2,60171.024152,58812
4,892,2023-06-11,1582,1910,1080,66390,6,62339.168024,58812


In [21]:
# adiciona as métricas técnicas ao dataframe, só teria estas métricas após ter a Target disponível, os dados em produção não teria esta Target store_sales para ser possível metrificar. 
# Só após ter ela dísponivel !
df['mae_model'] = df.apply(lambda x: mean_absolute_error(x['store_sales'], x['predict_store_sales']),axis=1) 
df['mape_model'] = df.apply(lambda x: mean_absolute_percentage_error(x['store_sales'], x['predict_store_sales']),axis=1) 
df['rmse_model'] = df.apply(lambda x: root_mean_squared_error(x['store_sales'], x['predict_store_sales']),axis=1) 

df['best_scenario_model'] = df['predict_store_sales'] + df['mae_model']
df['worst_scenario_model'] = df['predict_store_sales'] - df['mae_model']

# Calcular performance de negócio

In [22]:
calculate_model_revenue(df)

O baseline de média previu $-376,530.00 em relação as vendas reais.
O modelo previu $-46,429.78 em relação as vendas reais.
A diferença percentual entre o modelo e as vendas reais são de -0.42%
A diferença percentual entre o baseline de média e as vendas reais são de -3.43%
O modelo consegue recuperar $330,100.22 que eram perdidos ao usar a média como previsão de vendas!


# Carga da tabela final para o banco de dados (Supermarket_Silver)

In [23]:
df.head()

,store_id,sales_date,store_area,items_available,daily_customer_count,store_sales,store_cluster,predict_store_sales,average_store_sales,mae_model,mape_model,rmse_model,best_scenario_model,worst_scenario_model
0,896,2023-06-15,1174,1429,1110,54340,11,58334.967015,58812,3994.967015,0.073518,3994.967015,62329.934029,54340.000000
1,895,2023-06-14,1299,1560,770,96610,1,59284.845700,58812,37325.154300,0.386349,37325.154300,96610.000000,21959.691400
2,894,2023-06-13,1200,1436,1060,76440,11,58385.795468,58812,18054.204532,0.236188,18054.204532,76440.000000,40331.590935
3,893,2023-06-12,1387,1663,850,82080,2,60171.024152,58812,21908.975848,0.266922,21908.975848,82080.000000,38262.048303
4,892,2023-06-11,1582,1910,1080,66390,6,62339.168024,58812,4050.831976,0.061016,4050.831976,66390.000000,58288.336047


In [24]:
df.shape

(180, 14)

In [25]:
# Executar uma consulta SQL para verificar se a tabela já existe
cur.execute("""
    SELECT EXISTS (
        SELECT 1
        FROM   information_schema.tables 
        WHERE  table_name = 'Supermarket_Silver'
    );
""")

# Recuperar o resultado da consulta
table_exists = cur.fetchone()[0]

# Se a tabela não existir, criar a tabela
if not table_exists:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS Supermarket_Silver (
            store_id INT PRIMARY KEY,
            sales_date DATE,
            store_area INT,
            items_available INT,
            daily_customer_count INT,
            store_sales INT,
            store_cluster INT,
            predict_store_sales FLOAT,
            average_store_sales INT,
            mae_model FLOAT,
            mape_model FLOAT,
            rmse_model FLOAT,
            best_scenario_model FLOAT,
            worst_scenario_model FLOAT
        );
    """)
    conn.commit()

In [26]:
# Popular banco de dados

# Loop através das linhas do dataframe e inserir cada linha na tabela
for index, row in df.iterrows():
    cur.execute("""
        INSERT INTO Supermarket_Silver (store_id, sales_date, store_area, items_available, daily_customer_count, store_sales, store_cluster, predict_store_sales, average_store_sales,
        mae_model, mape_model, rmse_model, best_scenario_model, worst_scenario_model)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (
        int(row['store_id']),
        row['sales_date'],
        int(row['store_area']),
        int(row['items_available']),
        int(row['daily_customer_count']),
        int(row['store_sales']),
        int(row['store_cluster']),
        float(row['predict_store_sales']),
        int(row['average_store_sales']),
        float(row['mae_model']),
        float(row['mape_model']),
        float(row['rmse_model']),
        float(row['best_scenario_model']),
        float(row['worst_scenario_model'])
    ))

# Salvar as alterações no banco de dados
conn.commit()

# Fechar o cursor e a conexão com o banco de dados
cur.close()
conn.close()